<a href="https://colab.research.google.com/github/pkong0414/Deep-Learning-Projects/blob/master/Copy_of_clockAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing library

import keras
import os, shutil
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#use this when you use google colab only!
from google.colab import files, drive
import glob
import pandas as pd
import io
import numpy as np
import cv2
from PIL import Image
import pathlib

from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()


Using TensorFlow backend.


Created by: Patrick Kong, Cody Hawkings
                
    clockAI project:
    
    This clock ai would aim to use computervision in order to read a clock's time.
    It would be able to read digital and more importantly analog clocks.
    
    We will be using google colab for this project.

###Mounting Google Drive

In [2]:
#to import multiple files into google colab we first need to mount google drive
#so that we can make directory calls to our photos and CSV files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


####A quick check of the files in the directory to see if the drive mounted correctly

In [3]:
!ls "/content/drive/My Drive/Colab Notebooks/Clocks"

'All Clocks.zip'   temp		     test_digital	   test_val_digital.csv
'Analog Clock'	   test_analog	     test_digital.csv	   test_validation.csv
'Digital Clock'    test_analog.csv   test_val_analog.csv


####Directory Calls to Google Drive

In [109]:
#file directory for analog
base_A_dir = "/content/drive/My Drive/Colab Notebooks/Clocks/Analog Clock/train"

#file directory for digital
base_D_dir = "/content/drive/My Drive/Colab Notebooks/Clocks/Digital Clock/train"

#file directory for target (labels)
base_LA_dir = "/content/drive/My Drive/Colab Notebooks/Clocks/test_val_analog.csv"
base_LD_dir = "/content/drive/My Drive/Colab Notebooks/Clocks/test_val_digital.csv"


#open files from drive mount in google colab for correct picture names and lables
file_csv = open(base_LA_dir, 'r')
file2_csv = open(base_LD_dir, 'r')


#first we must go through the file and then split the information
#we then remove any unwanted fluff so that we are only left with 
#the filename and the picture label
#the filename will later be used to be joined with the correct directory
#so that we can open up pictures in order
train_A_labels = []
train_A_filenames = []
text_A = []

for files in file_csv:
  text_A = files.split( "," )
  train_A_labels.append( text_A[1].strip( '\n' ) )
  train_A_filenames.append( text_A[0].strip( '\ufeff' ) )
#print( train_A_labels )
# print( train_A_filenames )



train_D_labels = []
train_D_filenames = []
text_D = []

for files in file2_csv:
  text_D = files.split( "," )
  train_D_labels.append( text_D[1].strip( '\n' ) )
  train_D_filenames.append( text_D[0].strip( '\ufeff' ) )
#print( train_D_labels )
#print( train_D_filenames )


#Now that the filenames are in correct order we will append the 
#names of the pictures to their respective directory folder

temp2 = []
image_analog = []

for items in train_A_filenames:
  temp2 = os.path.join( base_A_dir, items )
  image_analog.append( temp2 )

print( image_analog[0] )

temp3 = []
image_digital = []

for item in train_D_filenames:
  temp3 = os.path.join( base_D_dir, item )
  image_digital.append( temp3 )

#print( image_digital[ : ] )

#as a sanity check we print out everything to make sure we are getting the 
#correct files in the correct order and to make sure that the filenames
#appended correctly with the directory paths

# AUTOTUNE = tf.data.experimental.AUTOTUNE


# x_input = tf.zeros( [ 720, 100, 100, 3], dtype = 'float32')

# def preprocess_image(image):
#   image = tf.image.decode_jpeg(image, channels=3)
#   image = tf.image.resize(image, [100, 100])
#   image /= 255.0  # normalize to [0,1] range
#   return image

# def load_and_preprocess_image(path):
#   image = tf.read_file(path)
#   x_input = preprocess_image( image )
#   return preprocess_image(image)


# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,8))
# for n,image in enumerate( train_A_filenames.take(2)):
#   plt.subplot(2,2,n+1)
#   plt.imshow(image)
#   plt.grid(True)
#   plt.xticks([])
#   plt.yticks([])
#   plt.show()
  
# plt.figure( figsize = ( 8, 8 ) )
# for n, image in enumerate( train_D_filenames.take( 2 ) ):
#   plt.subplot( 2, 2, n+1 )
#   plt.imshow( image )
#   plt.grid( True )
#   plt.xticks( [] )
#   plt.yticks( [] )
#   plt.show()
  
time = []
timeL = []
theta = np.zeros( ( 720, 2 ), dtype = 'float64' )
radius = np.zeros( ( 720, 2 ), dtype = 'float64' )
annotate_Hour = np.zeros( ( 720, 2 ), dtype = 'float64' )
annotate_Min = np.zeros( ( 720, 2 ), dtype = 'float64' )

hour = 0.0
minute = 0.0

#this will read from the csv labels and calculate the appropriate theta
#for the minute hand
  
#this will read the label and calculate the theta for the
#hour hand now
  
for i in range( len( train_A_labels ) ):
    
  time =  train_A_labels[ i ].split( ':' )
  hour = int( time[ 0 ] )
  minute = int( time[ 1 ] )
  
  annotate_Hour[ i ][ 1 ] = float( 0.5 * minute )
  annotate_Hour[ i ][ 0 ] = 2
  annotate_Min[ i ][ 1 ] = float( 6 * minute )
  annotate_Min[ i ][ 0 ] = 4
  
  
  timeL.append( str( time[ 0 ] + time[ 1 ] ) )
  
#this will stack theta in a 720 x 1 x 2 format
#printing output of both theta and radius
print( timeL[ :10 ] )

print( "Hour annotations: \n" )
print( annotate_Hour[ :5 ] )
print( "Minute annotations: \n" )
print( annotate_Min[ :5 ] )
  


# img_path = image_analog[0]
# img_path

# img_raw = tf.read_file(img_path)
# print(repr(img_raw))

# img_tensor = tf.image.decode_image(img_raw)

# print(img_tensor.shape)
# print(img_tensor.dtype)

# img_final = tf.image.resize_images(img_tensor, [150 , 150])
# img_final = img_final/255.0
# print(img_final.shape)
# print(img_final.numpy().min())
# print(img_final.numpy().max())


/content/drive/My Drive/Colab Notebooks/Clocks/Analog Clock/train/pic_1.JPG
['1200', '1201', '1202', '1203', '1204', '1205', '1206', '1207', '1208', '1209']
Hour annotations: 

[[2.  0. ]
 [2.  0.5]
 [2.  1. ]
 [2.  1.5]
 [2.  2. ]]
Minute annotations: 

[[ 4.  0.]
 [ 4.  6.]
 [ 4. 12.]
 [ 4. 18.]
 [ 4. 24.]]


In [0]:
label_to_index = dict((name, index) for index, name in enumerate(timeL))
list_values = [ v for v in label_to_index.values() ]
list_values
y_output = keras.utils.to_categorical(list_values, num_classes = 720)
print( y_output[0] )

In [0]:
#puts everything into a (720, 100, 100, 3) 4d tensor
imgs = []
for images in image_analog:
  img = image.load_img( images, target_size =( 100, 100 )  )
  image_tensor = image.img_to_array( img )
  image_tensor = np.expand_dims( image_tensor, axis = 0 )
  image_tensor /= 255.0
  imgs.append(image_tensor)
  
#imgs = np.concatenate(imgs, axis = 0)
print( imgs.shape )

plt.imshow( imgs[0] )
plt.show()

In [0]:
print( imgs.shape )
print( annotate_Hour.shape )
print( annotate_Min.shape )

x_input = tf.Tensor( [ imgs, annotate_Hour, annotate_Min ], [ 720, 100, 100, 3, 2, 2 ], dtype = ( 'float32', 'float64', 'float64' ) )

In [0]:
label_to_index = dict((name, index) for index, name in enumerate(timeL))
list_values = [ v for v in label_to_index.values() ]
list_values
y_output = keras.utils.to_categorical(list_values, num_classes = 720)
print( y_output[0] )

double checking length of the image after the resizing operation.

visualizing input in an array format.

printing length of image d after the resizing operation

In [0]:
k = 4
num_val_samples = sum([len(image_analog), len(image_digital)]) 
num_epochs = 100
num_scores = []

for i in range( k ):
  print( 'processing fold #', i )
  val_data = images[ i * num_val_samples: (i + 1) * num_val_samples ]
  val_targets = train_targets[ i * num_val_samples: (i + 1) * num_val_samples ]
  
#   partial_train_data = np.concatenate(
#     [ train_data[ :i * num_val_samples ],
#      train_data[ ( i + 1 ) * num_val_samples: ] ],
#     axis = 0 )
  
#   partial_train_targets = np.concatenate(
#     [ train_targets[ :i * num_val_samples ],
#      train_targets[ ( i + 1 ) * num_val_samples: ] ],
#     axis = 0 )
  
  

This is the k folding snippet we'll use to get our split our training and validation data.

Sanity test for the files.

In [0]:
#data gen

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#functions

def annotate_theta( img, labels ):
  #this will be a series of characters read from a label

  time = []
  theta = []
  radius = []
  hour = 0
  minute = 0
  
  
  #this will read from the csv labels and calculate the appropriate theta
  #for the minute hand
  
  #this will read the label and calculate the theta for the
  #hour hand now
  
  for i in range( len( labels ) ):
    
    time = labels[ i ].split( ':' )
    print( time )
    hour = int( time[ 0 ] )
    minute = int( time[ 1 ] )
    radius[ 0 ] = 2
    radius[ 1 ] = 4
    theta[ 0 ] = ( 0.5 * minute )
    theta[ 1 ] = ( 6 * minute )
  
  return radius, theta
  

def load_dataset(img, labels):
  X = []
  
  #once we annotated radius and theta into r and t
  #we will load it into a format of:
  #dataset = [ img, [ r(hour), r(minute) ], [t(hour), t(minute) ] ]
  
  for i in range( len( img ) ):
    r, t = annotate_theta( img[ i ], labels[ i ] )
    print( r )
    print( t )
    X.append( img[i] )
    X.append( r )
    X.append( t )
  
  return X


def residual_unit( inputs, filters, kernel, drop_out = 0.0, pooling = False ):
  res = inputs
  
  if pooling == True:
    res = conv2D( filters, kernel_size = 1, strides = ( 2, 2) )
    inputs = MaxPooling2d( pool_size = ( 2, 2 ) )
    
  inputs = BatchNormalization()( inputs )
  inputs = conv2D( filters, kernel, padding = 'same' )( inputs )
  inputs = activation( "relu" )( inputs )
  inputs = Dropout( drop_out )( inputs )
  inputs = BatchNormalization( inputs )
  inputs = conv2D( filters, kernel, padding = 'same' )( inputs )
  inputs = activation( "relu" )( inputs )
  
  inputs = keras.layers.add( [ inputs, res ] )
  
  return inputs

#main function


#using callback to save a clock AI on best validation.
#model.save('clock_ai_0.1.h5')

inputs = residual_unit( inputs, 32, 3 )
inputs = residual_unit( inputs, 32, 3 )

inputs = residual_unit( inputs, 64, 3, 0.0, True )
inputs = residual_unit( inputs, 64, 3 )

inputs = residual_unit( inputs, 128, 3, 0.0, True )
inputs = residual_unit( inputs, 128, 3 )

inputs = BatchNormalization()( inputs )
inputs = conv2D( 256, 3 )( inputs )
inputs = activation( "relu" )( inputs )

inputs = Flatten()
inputs = Dense( 720 )
inputs = activation( "softmax" )

model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy',
              metrics=["accuracy"])
model.fit( x_input , y_output, epochs=1, batch_size = 10, validation_data=(x_val, y_val) )


Gameplan is to use polar coordinates in order to give an additional feature to the learning model during training.

Initially considered using pixels to calculate theta since arctan( y / x ) will give us the angle of the clock hand.
However, since the pixels were blurry compounded by the fact that the conditions to make theta happen is complex, an alternative was considered.

Rather than use X and Y to calculate theta, the idea is then to use degree of 360 and divide by units of 60 ( which represent the minutes ). This gives us 6 degrees of movement per ***"minute"***.

For the ***"Hour"*** hand we know that the hand moves 5 minute units to move to the next hour. 5 * 6 gives us 30 degrees per hour, so if we take 30 degree /60 minutes we get 0.5 degrees/minute.

Same idea will be applied to the hour-hand cause this would be the one that gives the model problem since the hour-hand moves as the minute hand moves and makes things a little harder to do for the hour identification.

( 4/29/19 )

*added k folding so we can get our training and validation split accordingly for the small sample size

*added residual unit function which will allow us to make residual units with dropout and pooling capabilities

*added annotation function which will generate the other portion of input the network will be using to learn

*Data generator has been added but hasn't been customized to work with this problem

